In [ ]:
print("Hei!")

Hei!


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Multiply, ReLU, Lambda

In [ ]:
import keras.backend as k
import numpy as np
import sys, os, string, random

In [ ]:
characters = string.printable
char_indices = dict((c, i) for i, c in enumerate(characters))
indices_char = dict((i, c) for i, c in enumerate(characters))

In [ ]:
INPUT_VOCAB_SIZE = len(characters)
BATCH_SIZE = 200

In [ ]:
def encode_one_hot(line):
  x = np.zeros((len(line), INPUT_VOCAB_SIZE))
  for i, c in enumerate(line):
    index = char_indices[c] if c in characters else char_indices[' ']
    x[i][index] = 1
  return x

In [ ]:
def encode_values(line):
  x = np.zeros((len(line), INPUT_VOCAB_SIZE))
  for i, c in enumerate(line):
    index = char_indices[c] if c in characters else char_indices[' ']
    for a_c in characters:
      if a_c == c:
        x[i][index] = 1
      else:
        idx = char_indices[a_c]
        x[i][idx] = idx/index
    return x

In [ ]:
def decode_values(x):
  s = []
  for onehot in x:
    one_index = (np.abs(onehot - 1)).argmin()
    s.append(indices_char[one_index])
  return ''.join(s)

In [ ]:
def build_model():
  model = Sequential()
  model.add(Dense(1, input_shape=(INPUT_VOCAB_SIZE,)))
  model.add(Dense(INPUT_VOCAB_SIZE))
  return model

In [ ]:
def input_generator(nsamples):
  def generate_line():
    inline = []; outline = []
    for _ in range(nsamples):
      c = random.choice(characters)
      expected = c.lower() if c in string.ascii_letters else ' '
      inline.append(c); outline.append(expected)
    return ''.join(inline), ''.join(outline)

  while True:
    input_data, expected = generate_line()
    data_in = encode_one_hot(input_data)
    data_out = encode_values(expected)
    yield data_in, data_out

In [ ]:
def train(model):
  model.compile(loss='mse',
                optimizer='adam',
                metrics=['accuracy', 'mse'])
  print("compiled")

  input_gen = input_generator(BATCH_SIZE)
  validation_gen = input_generator(BATCH_SIZE)
  print("data is generated")

  model.fit(input_gen,
            epochs = 10, workers=1,
            steps_per_epoch = 1000,
            validation_data = validation_gen,
            validation_steps = 10)

In [ ]:
model = build_model()

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 101       
                                                                 
 dense_1 (Dense)             (None, 100)               200       
                                                                 
Total params: 301 (1.18 KB)
Trainable params: 301 (1.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
train(model)

compiled
data is generated
Epoch 1/10
1000/1000 [==============================] - 7s 6ms/step - loss: 0.0249 - accuracy: 0.0026 - mse: 0.0249 - val_loss: 0.0200 - val_accuracy: 0.0025 - val_mse: 0.0200
Epoch 2/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0264 - accuracy: 0.0039 - mse: 0.0264 - val_loss: 0.0249 - val_accuracy: 0.0050 - val_mse: 0.0249
Epoch 3/10
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0250 - accuracy: 0.0048 - mse: 0.0250 - val_loss: 0.0327 - val_accuracy: 0.0050 - val_mse: 0.0327
Epoch 4/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0279 - accuracy: 0.0048 - mse: 0.0279 - val_loss: 0.0180 - val_accuracy: 0.0050 - val_mse: 0.0180
Epoch 5/10
1000/1000 [==============================] - 4s 4ms/step - loss: 0.0259 - accuracy: 0.0049 - mse: 0.0259 - val_loss: 0.0535 - val_accuracy: 0.0045 - val_mse: 0.0535
Epoch 6/10
1000/1000 [==============================] - 4s 4ms/step - loss: 0.0249 - accuracy

In [ ]:
!wget https://www.gutenberg.org/cache/epub/46144/pg46144.txt

--2024-03-27 20:10:25--  https://www.gutenberg.org/cache/epub/46144/pg46144.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125798 (123K) [text/plain]
Saving to: ‘pg46144.txt’

pg46144.txt         100%[===================>] 122.85K   233KB/s    in 0.5s    

2024-03-27 20:10:27 (233 KB/s) - ‘pg46144.txt’ saved [125798/125798]



In [ ]:
!head pg46144.txt > pg46144_head.txt

In [ ]:
with open('pg46144_head.txt') as f:
  for line in f:
    if line.isspace(): continue
    batch = encode_one_hot(line)
    preds = model.predict(batch)
    normal = decode_values(preds)
    print(normal, line)

2/2 [==============================] - 0s 5ms/step
 ﻿The Project Gutenberg eBook of Six Cups of Coffee

3/3 [==============================] - 0s 4ms/step
 This ebook is for the use of anyone anywhere in the United States and

3/3 [==============================] - 0s 4ms/step
 most other parts of the world at no cost and with almost no restrictions

3/3 [==============================] - 0s 4ms/step
 whatsoever. You may copy it, give it away or re-use it under the terms

3/3 [==============================] - 0s 3ms/step
 of the Project Gutenberg License included with this ebook or online

3/3 [==============================] - 0s 3ms/step
